In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importing Libraries**

In [2]:
!pip install tensorflow-gpu==2.0.0-beta0
!pip install tensorflow_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.0.0-beta0 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1)
ERROR: No matching distribution found for tensorflow-gpu==2.0.0-beta0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [ ]:
import pandas as pd

# Increase precision of presented data for better side-by-side comparison
pd.set_option("display.precision", 8)

In [4]:
print("Version: ", tf.__version__)
print("Hub version: ", hub.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  2.8.2
Hub version:  0.12.0
Eager mode:  True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is NOT AVAILABLE


**Dataset Preparation**

In [5]:
data_root='/content/drive/MyDrive/Wayangs/'

In [6]:
train_path = '/content/drive/MyDrive/Wayangs/train'
test_path = '/content/drive/MyDrive/Wayangs/val'
IMAGE_SHAPE = (224, 224)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
)

valid_generator = valid_datagen.flow_from_directory(
    directory=test_path, 
    shuffle=True,
    target_size=IMAGE_SHAPE
)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    shear_range=0.2,
    rotation_range=30,
    fill_mode='reflect',
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
train_generator = train_datagen.flow_from_directory(
    directory=train_path,  
    shuffle=True,
    target_size=IMAGE_SHAPE)

Found 67 images belonging to 5 classes.
Found 446 images belonging to 5 classes.


In [7]:
image_batch_train, label_batch_train = next(iter(train_generator))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

Image batch shape:  (32, 224, 224, 3)
Label batch shape:  (32, 5)
['Wayang Beber' 'Wayang Gedog' 'Wayang Golek' 'Wayang Kulit'
 'Wayang Suluh']


**Model Training**

In [8]:
model = tf.keras.Sequential([
  hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", 
                 output_shape=[1280],
                 trainable=False),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1024, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])
model.build([None, 224, 224, 3])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 5)                 5125      
                                                                 
Total params: 3,574,853
Trainable params: 1,316,869
Non-trainable params: 2,257,984
_________________________________________________________________


In [9]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=['acc'])

In [10]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

hist = model.fit(
    train_generator, 
    epochs=35,
    verbose=1,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=val_steps_per_epoch).history

Epoch 1/35
14/14 [==============================] - 81s 6s/step - loss: 1.5660 - acc: 0.6839 - val_loss: 1.1705 - val_acc: 0.6119
Epoch 2/35
14/14 [==============================] - 25s 2s/step - loss: 0.6056 - acc: 0.7870 - val_loss: 1.0007 - val_acc: 0.6418
Epoch 3/35
14/14 [==============================] - 25s 2s/step - loss: 0.4903 - acc: 0.8274 - val_loss: 0.6878 - val_acc: 0.7164
Epoch 4/35
14/14 [==============================] - 25s 2s/step - loss: 0.4065 - acc: 0.8498 - val_loss: 0.6765 - val_acc: 0.7612
Epoch 5/35
14/14 [==============================] - 27s 2s/step - loss: 0.3698 - acc: 0.8632 - val_loss: 0.8043 - val_acc: 0.7164
Epoch 6/35
14/14 [==============================] - 25s 2s/step - loss: 0.3321 - acc: 0.8812 - val_loss: 0.8929 - val_acc: 0.7313
Epoch 7/35
14/14 [==============================] - 25s 2s/step - loss: 0.3432 - acc: 0.8834 - val_loss: 0.8378 - val_acc: 0.7313
Epoch 8/35
14/14 [==============================] - 25s 2s/step - loss: 0.3542 - acc: 0.86

In [11]:
final_loss, final_accuracy = model.evaluate(valid_generator, steps = val_steps_per_epoch)
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

3/3 [==============================] - 3s 532ms/step - loss: 0.4578 - acc: 0.8060
Final loss: 0.46
Final accuracy: 80.60%


**Saving Model**

In [12]:
WAY_SAVED_MODEL = "saved_models/wayang"
tf.keras.models.save_model(model, WAY_SAVED_MODEL)

INFO:tensorflow:Assets written to: saved_models/wayang/assets


INFO:tensorflow:Assets written to: saved_models/wayang/assets


**Converting Model to TFlite**

In [13]:
way_model = tf.keras.models.load_model(WAY_SAVED_MODEL,custom_objects={'KerasLayer':hub.KerasLayer})

In [15]:
converter = tf.lite.TFLiteConverter.from_saved_model(WAY_SAVED_MODEL)
tflite_model = converter.convert()

open("model_wayang.tflite", "wb").write(tflite_model)

14135072

In [19]:
from shutil import copyfile

In [22]:
saved_model_path = "/content/model_wayang.tflite"
gdrive_saved_model_path = '/content/drive/MyDrive/Wayang Capstone/ModelTFlite'
copyfile(saved_model_path, gdrive_saved_model_path)

'/content/drive/MyDrive/Wayang Capstone/ModelTFlite'